# Bias Buccaneers Image Recognition Challenge: Unsupervised Solution

In [1]:
#import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image
#import K-Means
from sklearn.cluster import KMeans
# important metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Prepare the data

### Data Pre-processing

In [3]:
dataset_path = "kaggle/input/bias-buccaneers"
train_df = pd.read_csv(f"/{dataset_path}/train/labels.csv")
test_df = pd.read_csv(f"/{dataset_path}/test/labels.csv")

In [4]:
#there are around 5 grayscale images in the train dataset, so it is better to remove them.
grayscale_img_indices = [226, 936, 7019, 8895, 9193]
train_df.drop(labels=grayscale_img_indices, axis=0, inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [5]:
print(f"Number of Training samples: {len(train_df)}")
print(f"Number of Testing samples: {len(test_df)}")

Number of Training samples: 12278
Number of Testing samples: 3000


In [6]:
categories = train_df.columns[1:].tolist()
print(categories)

['skin_tone', 'gender', 'age']


Now we need to encode the labels into an integer such that we can use PyTorch's CrossEntropyLoss.

In [7]:
skin_tone_labels = [f"monk_{i}" for i in range(1,11)]
gender_labels = ["male", "female"]
age_labels = ["0_17", "18_30", "31_60", "61_100"]

In [8]:
#encode train samples
train_df['skin_tone'].replace(skin_tone_labels, list(range(len(skin_tone_labels))), inplace=True)
train_df['gender'].replace(gender_labels, list(range(len(gender_labels))), inplace=True)
train_df['age'].replace(age_labels, list(range(len(age_labels))), inplace=True)

In [9]:
#encode test samples
test_df['skin_tone'].replace(skin_tone_labels, list(range(len(skin_tone_labels))), inplace=True)
test_df['gender'].replace(gender_labels, list(range(len(gender_labels))), inplace=True)
test_df['age'].replace(age_labels, list(range(len(age_labels))), inplace=True)

In [10]:
#train_df.isnull().sum()
#3730 unlabeled images are there

In [11]:
#train_df still has null values for labels, uncomment and run this cell to get only the labelled images
train_df_labeled = train_df[train_df["skin_tone"].notna()].copy(deep=True) # take only labeled data
#Modify the index values of the df
train_df_labeled.reset_index(drop=True, inplace=True)
#print(f"No.of.Labeled images: {len(train_df_labeled)}")
#12278-3730 = 8548

### Building the Dataset

In [12]:
class ImageDataset(Dataset):
    def __init__(self, df, data_path, image_transform):
        self.df = df
        self.data_path = data_path
        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img = Image.open(f"{self.data_path}{self.df['name'][index]}")
        if self.image_transform:
            img = self.image_transform(img)

        #we need to provide labels for skin_tone, gender, age
        labels = (self.df['skin_tone'][index], self.df['gender'][index], self.df['age'][index])
        return img, labels

In [13]:
#find index values for grayscale images

#indices = []
#for i in range(len(dataset)):
#    img, _ = dataset[i]
#    if (img.shape[0]==1):
#        indices.append(i)
#print(indices)

## Helper functions

In [14]:
def extract_features(dataset:torch.utils.data.Dataset, feature_extractor:torch.nn.Module):
    features = []
    num_samples = len(dataset)
    for i in range(num_samples):
        img,_ = dataset[i]
        img = img.unsqueeze(0)
        img = img.to(device)
        img = feature_extractor(img)
        img = torch.reshape(img, (-1,))
        img = img.detach().cpu().numpy()
        features.append(img)
    return np.array(features)

In [15]:
def make_predictions(data_features, model):
    predictions = model.predict(data_features)
    return predictions

In [16]:
# Since K-Means Does not know about our labels we have to change labels of k-means according to our usage

# mapping labels from cluster to original labels
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # For loop to run through each label of cluster label
    for i in range(len(np.unique(clusters))):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1]).argmax()
        reference_label[i] = num
    return reference_label

# Mapping predictions to original labels
def get_labels(clusters,refernce_labels):
    temp_labels = np.random.rand(len(clusters))
    for i in range(len(clusters)):
        temp_labels[i] = reference_labels[clusters[i]]
    return temp_labels

## Extract features using Resnet

In [17]:
from torchvision.models import resnet18

In [18]:
class Feature_Extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.alexnet(pretrained=True)
        self.model.avgpool = nn.Identity()
        self.model.classifier = nn.Identity()
    def forward(self, x):
        x = self.model(x)
        x = torch.reshape(x, (-1,))
        return x

In [19]:
feature_extractor = Feature_Extractor().to(device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

## Train the Model

In [20]:
image_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
dataset = ImageDataset(train_df_labeled, f"/{dataset_path}/train/",image_transform)

In [21]:
image_features = extract_features(dataset, feature_extractor)
print(image_features.shape)

(8548, 12544)


In [22]:
skintone_model = KMeans(n_clusters=10)
skintone_model.fit(image_features)

KMeans(n_clusters=10)

In [23]:
gender_model = KMeans(n_clusters=3)
gender_model.fit(image_features)

KMeans(n_clusters=3)

In [24]:
age_model = KMeans(n_clusters=4)
age_model.fit(image_features)

KMeans(n_clusters=4)

## Saving the trained models

In [25]:
os.mkdir("saved models")

In [26]:
import pickle

In [27]:
pickle.dump(skintone_model, open("./saved models/skintone_model.pkl", "wb"))
pickle.dump(gender_model, open("./saved models/gender_model.pkl", "wb"))
pickle.dump(age_model, open("./saved models/age_model.pkl", "wb"))

---